# Segmentation Example
> Train a U-Net for pixelwise segmentation of the prostate

In [1]:
import monai
import ignite

from prostate158.utils import load_config
from prostate158.train import SegmentationTrainer
from prostate158.report import ReportGenerator
from prostate158.viewer import ListViewer

### Installation Procedure of dependencies 

In [2]:
# # if you have a requirements.txt:
# !pip install --upgrade pip
# !pip install -r requirements.txt

# # Otherwise install core libs directly:
# !pip install monai["all"] ignite matplotlib pyyaml munch

# Finally
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python
# !pip install ipywidgets

In [2]:
import torch

print(torch.cuda.is_available())  # For PyTorch

True


All parameters needed for training and evaluation are set in `anatomy.yaml` file. 

In [3]:
cfg = load_config("anatomy.yaml")  # change to 'anatomy.yaml' for anatomy segmentation
# cfg = load_config("tumor.yaml")
monai.utils.set_determinism(seed=cfg.seed)
cfg.model.type = "rrunet3d"
# cfg.model.type = "unet"

In [4]:
from prostate158.model import get_model

model = get_model(cfg).to(cfg.device)  # Move the model to the selected device
print(model)

[get_model] model.type = 'rrunet3d'
RRUNet3D(
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unpool): Upsample(scale_factor=2.0, mode='nearest')
  (input_conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (output_conv): Conv3d(16, 3, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
  (output_activation): Softmax(dim=1)
  (encoders): ModuleList(
    (0): ResidualBlock(
      (CONV): ConvBlock(
        (ops): ModuleList(
          (0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (RCNN): Sequential(
        (0): RecurrentBlock(
          (conv): Sequential(
            (0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_s

Create supervised trainer for segmentation task

In [5]:
trainer = SegmentationTrainer(
    progress_bar=True,
    early_stopping=True,
    metrics=["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics=True,
    config=cfg,
)

# # Load pre-trained weights from tumor.pt
trainer.load_checkpoint("models/network_anatomy_9_key_metric_205_epoch=0.6790.pt")

[get_model] model.type = 'rrunet3d'
Loading checkpoint from models/network_anatomy_9_key_metric_205_epoch=0.6790.pt to device cuda:0


Adding a learning rate scheduler for one-cylce policy. 

In [6]:
trainer.fit_one_cycle()

Let's train. This can take several hours. 

In [ ]:
trainer.run()

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


2025-08-11 18:32:27,145 - INFO - Epoch: 1/500, Iter: 1/119 -- train_loss: 1.2907 


[1/119]   1%|           [00:00<?]

2025-08-11 18:32:27,397 - INFO - Epoch: 1/500, Iter: 2/119 -- train_loss: 1.4039 
2025-08-11 18:32:27,645 - INFO - Epoch: 1/500, Iter: 3/119 -- train_loss: 1.5261 
2025-08-11 18:32:27,878 - INFO - Epoch: 1/500, Iter: 4/119 -- train_loss: 1.3336 
2025-08-11 18:32:28,095 - INFO - Epoch: 1/500, Iter: 5/119 -- train_loss: 1.2843 
2025-08-11 18:32:30,154 - INFO - Epoch: 1/500, Iter: 6/119 -- train_loss: 1.4816 
2025-08-11 18:32:30,410 - INFO - Epoch: 1/500, Iter: 7/119 -- train_loss: 1.4172 
2025-08-11 18:32:30,628 - INFO - Epoch: 1/500, Iter: 8/119 -- train_loss: 1.3695 
2025-08-11 18:32:30,855 - INFO - Epoch: 1/500, Iter: 9/119 -- train_loss: 1.4262 
2025-08-11 18:32:31,078 - INFO - Epoch: 1/500, Iter: 10/119 -- train_loss: 1.4341 
2025-08-11 18:32:31,304 - INFO - Epoch: 1/500, Iter: 11/119 -- train_loss: 1.3810 
2025-08-11 18:32:31,528 - INFO - Epoch: 1/500, Iter: 12/119 -- train_loss: 1.3260 
2025-08-11 18:32:31,778 - INFO - Epoch: 1/500, Iter: 13/119 -- train_loss: 1.3980 
2025-08-11 1

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


...

[1/20]   5%|5          [00:00<?]

2025-08-12 01:11:39,642 - INFO - Epoch: 137/500, Iter: 1/119 -- train_loss: 1.3957 


[1/119]   1%|           [00:00<?]

2025-08-12 01:11:39,867 - INFO - Epoch: 137/500, Iter: 2/119 -- train_loss: 1.3537 
2025-08-12 01:11:40,482 - INFO - Epoch: 137/500, Iter: 3/119 -- train_loss: 1.2463 
2025-08-12 01:11:40,707 - INFO - Epoch: 137/500, Iter: 4/119 -- train_loss: 1.3158 
2025-08-12 01:11:40,937 - INFO - Epoch: 137/500, Iter: 5/119 -- train_loss: 1.2725 
2025-08-12 01:11:41,194 - INFO - Epoch: 137/500, Iter: 6/119 -- train_loss: 1.3135 
2025-08-12 01:11:41,411 - INFO - Epoch: 137/500, Iter: 7/119 -- train_loss: 1.2416 
2025-08-12 01:11:41,643 - INFO - Epoch: 137/500, Iter: 8/119 -- train_loss: 1.4404 
2025-08-12 01:11:42,252 - INFO - Epoch: 137/500, Iter: 9/119 -- train_loss: 1.3258 
2025-08-12 01:11:42,490 - INFO - Epoch: 137/500, Iter: 10/119 -- train_loss: 1.2840 
2025-08-12 01:11:43,495 - INFO - Epoch: 137/500, Iter: 11/119 -- train_loss: 1.2866 
2025-08-12 01:11:44,231 - INFO - Epoch: 137/500, Iter: 12/119 -- train_loss: 1.4798 
2025-08-12 01:11:44,487 - INFO - Epoch: 137/500, Iter: 13/119 -- train_lo

[1/20]   5%|5          [00:00<?]

Finish the training with final evaluation of the best model. To allow visualization of all outputs, add OutputStore handler first. Otherwise only output form the last epoch will be accessible. 

In [ ]:
eos_handler = ignite.handlers.EpochOutputStore()
eos_handler.attach(trainer.evaluator, "output")

In [ ]:
trainer.evaluate(checkpoint="models/<model-name>.pt")

Generate a markdown document with segmentation results

In [ ]:
report_generator = ReportGenerator(config.run_id, config.out_dir, config.log_dir)
report_generator.generate_report()

Have a look at some outputs

In [ ]:
output = trainer.evaluator.state.output
keys = ["image", "label", "pred"]
outputs = {k: [o[0][k].detach().cpu().squeeze() for o in output] for k in keys}

In [ ]:
ListViewer(
    [o.transpose(0, 2).flip(-2) for o in outputs["image"][0:3]]
    + [o.argmax(0).transpose(0, 2).flip(-2).float() for o in outputs["label"][0:3]]
    + [o.argmax(0).transpose(0, 2).flip(-2).float() for o in outputs["pred"][0:3]]
).show()